In [1]:
!pip install tensorflow transformers

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00


In [5]:
!pip install huggingface_hub


In [3]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report
from huggingface_hub import notebook_login
from datasets import load_dataset



In [4]:

# Load the dataset
dataset = load_dataset("gretelai/symptom_to_diagnosis")


# Convert train_dataset to DataFrame
df_train = pd.DataFrame(dataset["train"])
df_test = pd.DataFrame(dataset["test"])

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
df_train['input_text'] = df_train['input_text'].astype(str)
df_test['input_text'] = df_test['input_text'].astype(str)

# Encode labels using LabelEncoder
label_encoder = LabelEncoder()
df_train['encoded_labels'] = label_encoder.fit_transform(df_train['output_text'])
df_test['encoded_labels'] = label_encoder.transform(df_test['output_text'])

# Split the data into train and validation sets
train_df, val_df = train_test_split(df_train, test_size=0.1, stratify=df_train['output_text'])

# Tokenize the text data and apply preprocessing
max_length = 150
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenized_train = tokenizer(list(train_df['input_text']), padding=True, truncation=True, max_length=max_length, return_tensors="tf")
tokenized_val = tokenizer(list(val_df['input_text']), padding=True, truncation=True, max_length=max_length, return_tensors="tf")

# Convert labels to one-hot encoded format
Y_train_encoded = tf.keras.utils.to_categorical(train_df['encoded_labels'], num_classes=len(label_encoder.classes_))
Y_val_encoded = tf.keras.utils.to_categorical(val_df['encoded_labels'], num_classes=len(label_encoder.classes_))

# Create TensorFlow datasets
BATCH_SIZE = 64
train_dataset = tf.data.Dataset.from_tensor_slices((dict(tokenized_train), Y_train_encoded)).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((dict(tokenized_val), Y_val_encoded)).batch(BATCH_SIZE)

# Define the model
num_classes = len(label_encoder.classes_)
int2label = dict(enumerate(label_encoder.classes_))
label2int = {v: k for k, v in int2label.items()}

model = TFAutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased",
    num_labels=num_classes,
    id2label=int2label,
    label2id=label2int,
    output_attentions=True
)

model.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-5),
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20
)

# Evaluate the model on the test set
tokenized_test = tokenizer(list(df_test['input_text']), padding=True, truncation=True, max_length=max_length, return_tensors="tf")
Y_test_encoded = tf.keras.utils.to_categorical(df_test['encoded_labels'], num_classes=num_classes)

test_dataset = tf.data.Dataset.from_tensor_slices((dict(tokenized_test), Y_test_encoded)).batch(BATCH_SIZE)

test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

# Print the classification report for more detailed metrics
predictions = model.predict(test_dataset)
predicted_labels = tf.argmax(predictions.logits, axis=1)
true_labels = tf.argmax(tf.constant(Y_test_encoded), axis=1)

print(classification_report(true_labels, predicted_labels, target_names=label_encoder.classes_))

notebook_login()
model.push_to_hub("Zabihin/Symptom_to_Diagnosis", use_auth_token="********")
tokenizer.push_to_hub("Zabihin/Symptom_to_Diagnosis", use_auth_token="****************")

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

...

Epoch 1/20
12/12 [==============================] - 54s 589ms/step - loss: 3.0795 - accuracy: 0.0743 - val_loss: 2.9457 - val_accuracy: 0.1628
Epoch 2/20
12/12 [==============================] - 4s 318ms/step - loss: 2.7918 - accuracy: 0.2529 - val_loss: 2.5632 - val_accuracy: 0.3256
Epoch 3/20
12/12 [==============================] - 4s 319ms/step - loss: 2.3844 - accuracy: 0.5359 - val_loss: 2.1057 - val_accuracy: 0.6744
Epoch 4/20
12/12 [==============================] - 4s 320ms/step - loss: 1.9503 - accuracy: 0.7601 - val_loss: 1.7382 - val_accuracy: 0.7442
Epoch 5/20
12/12 [==============================] - 4s 319ms/step - loss: 1.5516 - accuracy: 0.8370 - val_loss: 1.3624 - val_accuracy: 0.7674
Epoch 6/20
12/12 [==============================] - 4s 319ms/step - loss: 1.2056 - accuracy: 0.9035 - val_loss: 1.0704 - val_accuracy: 0.8488
Epoch 7/20
12/12 [==============================] - 4s 319ms/step - loss: 0.9322 - accuracy: 0.9531 - val_loss: 0.8703 - val_accuracy: 0.8372
Epoch

/usr/local/lib/python3.10/dist-packages/transformers/modeling_tf_utils.py:3104: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tf_model.h5:   0%|          | 0.00/434M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:853: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/Zabihin/Symptom_to_Diagnosis/commit/c629fbc8f5c37059ee33d144c0005a838f11a0bc', commit_message='Upload tokenizer', commit_description='', oid='c629fbc8f5c37059ee33d144c0005a838f11a0bc', pr_url=None, pr_revision=None, pr_num=None)